###### ドライブ共有準備

In [0]:
from google.colab import drive
drive.mount('./gdrive')

In [0]:
import sys
sys.getdefaultencoding()



---



# Kerasで文章分類器を作成
*   Kerasはtensorflowなどで動くPythonで書かれたニューラルネットワークライブラリ。詳細は[こちら](https://keras.io/ja/)。
*   データは「livedoor ニュースコーパス」を使用。各記事ファイルにはクリエイティブ・コモンズライセンス「表示 – 改変禁止」が適用される。詳細はフォルダのLICENSE.txt参照。


### データの事前展開
> Google DriveからColabへのファイルコピー

In [0]:
!pip install -U -q PyDrive

In [0]:
# Google Driveアクセス準備
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Google Driveからのコピー
id = '1BwbaNc3TJu5ENE36n3EOty8a-Z6XXe33'  # 共有リンクで取得した id= より後の部分(ldcc-20140209.tar.gzの例)
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('ldcc-2014-2-9.zip')
# 終わったら、ファイル画面の更新をして確認

In [0]:
# tarの展開（中身はZipじゃないみたいなので展開だけでOK）
# 終わったら、ファイル画面の更新をして/textがあればOK
#!tar -xvf ldcc-20140209.tar.gz
!unzip ldcc-2014-2-9.zip

### ツールの準備（janome）

In [0]:
# janomeインストール：形態素解析エンジン。形態素の分割、品詞判定、分かち書きができる
!pip install janome

In [0]:
# janomeの動作確認
from janome.tokenizer import Tokenizer

# テスト
word = Tokenizer().tokenize('患者さんは診察後に支払窓口で会計をすることなく、帰宅が可能となります')
for i in range(len(word)):
    print(i,word[i])

### データ準備（ベクトル化）

日本語テキストを機械学習に入力するため、テキストを数値化（ベクトル化）する。

ベクトル化する前に、**janomeで「分かち書き」したデータを作る**　

※分かち書きとは、英語のような「語を空白で区切ったもの」。今回はシンプルに名詞のみ抽出し、その他は捨てる。


In [0]:
from janome.tokenizer import Tokenizer
import os, glob
 
# Janomeを使って形態素解析
ja_tokenizer=Tokenizer()
 
# 分かち書きし、日本語文から名詞のみ抽出する
def ja_tokenize(text):
    res=[]
    lines=text.split("\n")
    lines=lines[3:] # サンプルデータの最初の3行はヘッダーと題名なので捨てる
    for line in lines:
        malist=ja_tokenizer.tokenize(line)
        for tok in malist:
            ps=tok.part_of_speech.split(",")[0]
            # 名詞でなければ以下の処理をスキップ
            if not ps in ['名詞']: continue               
            w=tok.base_form
            if w=="*" or w=="": w=tok.surface
            if w=="" or w=="\n": continue
            res.append(w)
        res.append("\n")
    return res
 
# テキストデータを読み込み
dir ='./text'
for path in glob.glob(dir + "/*/*.txt", recursive=True):
    # LICENSE.txtは以下の処理をスキップ
    if path.find("LICENSE")>0:continue          
    #print(path)
    path_wakati=path + ".wakati"
    # 既に "wakati"ファイルがあれば以下の処理をスキップ
    if os.path.exists(path_wakati):continue        
    text=open(path,"r", encoding='utf-8').read() 
    words=ja_tokenize(text)
    wt=" ".join(words)
    open(path_wakati, "w", encoding="utf-8").write(wt)
    

In [0]:
# 分かち書き処理の確認
print("---------- ↓↓↓↓ 分かち書き前 ↓↓↓↓ ------------")
from __future__ import with_statement
with open('./text/0/dokujo-tsushin-4778031.txt', 'r') as f:
    print(f.read())
print("---------- ↓↓↓↓ 分かち書き後 ↓↓↓↓ ------------")
from __future__ import with_statement
with open('./text/0/dokujo-tsushin-4778031.txt.wakati', 'r') as f:
    print(f.read())

In [0]:
# 分かち書きしたテキストを数値データに変換  ※15-16分かかる※
from sklearn.model_selection import train_test_split
import numpy as np
import os, glob

dic={} # 辞書初期化
count = 0  # 辞書登録用カウンタ
folder = ["0","1","2","3","4,","5","6","7","8"]
#folder = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme","movie-enter","peachy","smax","sports-watch","topic-news"]
x, y = [], []

for index, name in enumerate(folder):
    dir = "./text/" + name
    files = sorted(glob.glob(dir + "/*.wakati"))  # ソートして読み込み
    for i, file in enumerate(files):
        with open(file, "r", encoding='utf-8') as f:
             text = f.read()
             text = text.strip()
             text = text.split(" ")
             result = []  # 単語を数字化した結果を入れるリスト
             for word in text:
                 word = word.strip()  
                 if word == "": continue
                 if not word in dic: # 未登録の場合
                    dic[word] = count  # count の数字で辞書に登録
                    num = count
                    count +=1
                    #print(num,word)  # 数字と単語を表示
                 else:
                    num=dic[word] # 数字を辞書で調べる
                 result.append(num)  # リストに数字を追加
             x.append(result)  # Xの作成：文書に含まれる名詞を数値(辞書内の番号)に置き換えたデータ。名詞と数値の対応関係は辞書(dic)で管理。
             y.append(index)   # Yの作成：文書のカテゴリ番号を追加。0 はdokujo-tsushin, 1 はit-life-hackなど

             
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

＜参考＞ベクトル化したデータの確認

In [0]:
# 2番目のテキスト文書の表示（ベクトル化 前）
from __future__ import with_statement
with open('./text/0/dokujo-tsushin-4778031.txt.wakati', 'r') as f:
    print(f.read())

In [0]:
# 先頭の単語に対応した辞書登録番号(数字)を表示
print(dic['携帯'])
print(dic['電話'])
print(dic['普及'])

In [0]:
# ベクトル化した文書を表示
print(x[1])  # 2番目(0,1,2,...)のテキスト(dokujo-tsushin-4778031.txt.wakati)を数値化したもの
print(y[1])  # 2番目(0,1,2,...)のテキストのカテゴリ番号(dokujo-tsushinは0)

In [0]:
# （参考）辞書に登録した単語と登録番号の内容
print(dic)

In [0]:
x_train[0]

### 学習モデルの構築

In [0]:
from __future__ import print_function
import numpy as np
import keras
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

max_words = 1000  # ワード数を増やすと精度が若干かわる
batch_size = 32
epochs = 10

print('-------分かち書き後のデータのベクトル化-------')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('-------モデル構築-------')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.save('MyNLCsam.h5')
history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.1)

**モデルを評価**

In [0]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)     # 損失値の計算
print('Test score:', score[0])       # 損失値(1-損失値)
print('Test accuracy:', score[1])    # 精度
model.save('MyNLCsam.h5')

### 学習経過のプロット
import matplotlib.pyplot as plt
from IPython.display import Image,display_png
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label = "Training acc" )
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()
plt.show()
plt.close()

### 推論のテスト

とりあえず、ベクトル化したテキストが既に準備されている前提で、推論のテスト実施。

In [0]:
# 推論したい文章を入力
path_w = 'in-sample.txt'
sam = 'AIの安心・安全な利用に向けた「富士通グループAIコミットメント」を策定　\
当社は、このたび、AIの安心・安全な利用に向けて、当社グループにおけるAI関連の技術や\
ソリューション・サービスの研究開発に際し、AI倫理を含む価値観をまとめた、\
「富士通グループAIコミットメント」を策定しました。\
当社グループは、本コミットメントに基づき、「人と協調する、人を中心とするAI」を具現化した\
技術やソリューション・サービスの提供をグローバルに推進していきます。'

with open(path_w, mode='w') as f:    # 推論したいテキストを作成
    f.write(sam)

print('---------推論文章（分かち書き前）------------')
with open(path_w) as f:
    print(f.read())


# ----分かち書きして、FileName.txt.wakatiで書き込み----
from janome.tokenizer import Tokenizer
import os, glob
ja_tokenizer=Tokenizer()    # 分かち書きし、日本語文から名詞のみ抽出する

def ja_tokenize(text):
    res=[]
    lines=text.split("\n")
    for line in lines:
        malist=ja_tokenizer.tokenize(line)
        for tok in malist:
            ps=tok.part_of_speech.split(",")[0]
            # 名詞でなければ以下の処理をスキップ
            if not ps in ['名詞']: continue               
            w=tok.base_form
            if w=="*" or w=="": w=tok.surface
            if w=="" or w=="\n": continue
            res.append(w)
        res.append("\n")
    return res
 
for path in glob.glob(path_w, recursive=True):
    path_wakati=path + ".wakati"
    # 既に "wakati"ファイルがあれば以下の処理をスキップ
    if os.path.exists(path_wakati):continue        
    text=open(path,"r", encoding='utf-8').read() 
    words=ja_tokenize(text)
    wt=" ".join(words)
    open(path_wakati, "w", encoding="utf-8").write(wt)

print('')
print('---------推論文章（分かち書き後）------------')
with open(path_wakati) as f:
    print(f.read())


# ----分かち書きしたテキストを数値データに変換----
import numpy as np
import os, glob
#dic={} # 既存辞書に追加するので初期化しない
#count = 0  # 既存辞書登録用カウンタに追加するので初期化しない
x1 = []
for i, file in enumerate(files):
    with open(path_wakati, "r", encoding='utf-8') as f:
         text = f.read()
         text = text.strip()
         text = text.split(" ")
         result = []  # 単語を数字化した結果を入れるリスト
         for word in text:
             word = word.strip()  
             if word == "": continue
             if not word in dic: # 未登録の場合
                dic[word] = count  # count の数字で辞書に登録
                num = count
                count +=1
             else:
                num=dic[word] # 数字を辞書で調べる
             result.append(num)  # リストに数字を追加
             x1.append(result)  # Xの作成：文書に含まれる名詞を数値(辞書内の番号)に置き換えたデータ。名詞と数値の対応関係は辞書(dic)で管理。
print('---------推論文章（ベクトル化後）------------')
print(x1)             
x1 = tokenizer.sequences_to_matrix(x1, mode='binary')


model.load_weights('MyNLCsam.h5')
print('')
print('---------！！推論結果！！------------')
pred = model.predict(x1, batch_size=1, verbose=0)
pred_score = np.max(pred)
pred_label = np.argmax(pred)
print('予測スコア:', pred_score * 100 , '%')       # 
print('予測ラベル:', pred_label)    # 

### （その他）メモ書きなど

*  notebook上で**デバッガー**を使う際は[ここを参照](https://recruit-tech.co.jp/blog/2018/10/16/jupyter_notebook_tips/#b3)

In [0]:
#辞書型の使用例
def getValue(key, items):
    values = [x['Value'] for x in items if 'Key' in x and 'Value' in x and x['Key'] == key]
    return values[0] if values else None


# 使用例
items = [{'Key': 0,'Value': '独女通信'},
         {'Key': 1,'Value': 'ITライフハック'},
         {'Key': 2,'Value': '家電チャンネル'},
         {'Key': 3,'Value': 'Livedoor home'},
         {'Key': 4,'Value': 'movie-enter'},
         {'Key': 5,'Value': 'Peachy'},
         {'Key': 6,'Value': 'Smax'},
         {'Key': 7,'Value': 'スポーツウォッチ'},
         {'Key': 8,'Value': 'トピックニュース'}]
print(getValue(3, items))

---